In [ ]:
%matplotlib inline


# Retieving wind drift factor from trajectory


In [ ]:
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from opendrift.models.oceandrift import OceanDrift
from opendrift.models.physics_methods import wind_drift_factor_from_trajectory

A very simple drift model is: current + wind_drift_factor*wind
where wind_drift_factor for surface drift is typically
0.035 if Stokes drift included, and 0.02 in addition to Stokes drift.
This example illustrates how a best-fit wind_drift_factor
can be calculated from an observed trajectory.



First we simulate a synthetic drifter trajectory



In [ ]:
ot = OceanDrift(loglevel=50)
ot.add_readers_from_list([ot.test_data_folder() +
    '16Nov2015_NorKyst_z_surface/norkyst800_subset_16Nov2015.nc',
    ot.test_data_folder() + '16Nov2015_NorKyst_z_surface/arome_subset_16Nov2015.nc'], lazy=False)

Using a wind_drift_factor of 0.33 i.e. drift is current + 3.3% of wind speed



In [ ]:
ot.seed_elements(lon=4, lat=60, number=1, time=ot.readers[list(ot.readers)[0]].start_time,
        wind_drift_factor=0.033)

Adding some horizontal diffusivity as "noise"



In [ ]:
ot.set_config('drift:horizontal_diffusivity', 10)
ot.run(duration=timedelta(hours=12), time_step=600)

ot.plot(fast=True, title='Synthetic drifter trajectory')

Secondly, calculating the wind_drift_factor which reproduces the "observed" trajectory with minimal difference



In [ ]:
drifter_lons = ot.history['lon'][0]
drifter_lats = ot.history['lat'][0]
drifter_times = ot.get_time_array()[0]

o = OceanDrift(loglevel=50)
o.add_readers_from_list([o.test_data_folder() +
    '16Nov2015_NorKyst_z_surface/norkyst800_subset_16Nov2015.nc',
    o.test_data_folder() + '16Nov2015_NorKyst_z_surface/arome_subset_16Nov2015.nc'], lazy=False)
t = o.get_variables_along_trajectory(variables=['x_sea_water_velocity', 'y_sea_water_velocity', 'x_wind', 'y_wind'],
        lons=drifter_lons, lats=drifter_lats, times=drifter_times)

wind_drift_factor = wind_drift_factor_from_trajectory(t)
print(wind_drift_factor)

The mean retrieved wind_drift_factor is 0.036, slichtly higher than the value 0.033 used for the simulation.
The difference is due to the "noise" added by horizontal diffusion.
Note that the retieved wind_drift_factor is linked to the wind and current used for the retrieval,
other forcing datasets will yeld different value of the wind_drift_factor.



In [ ]:
print(wind_drift_factor.mean())

plt.hist(wind_drift_factor, label='Retrieved wind_drift_factor')
plt.axvline(x=0.033, label='Actual wind_drift_factor of 0.033', color='k')
plt.axvline(x=wind_drift_factor.mean(), label='Mean retieved wind_drift_factor of %.3f' % wind_drift_factor.mean(), color='r')
plt.ylabel('Number')
plt.xlabel('Wind drift factor  [fraction]')
plt.legend()
plt.show()